In [3]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.metrics import silhouette_score, r2_score, mean_squared_error, mean_absolute_error

In [4]:
"""
Implementation of logistic ordinal regression (aka proportional odds) model
"""

from __future__ import print_function

from sklearn import metrics
from scipy import linalg, optimize, sparse
import numpy as np
import warnings

In [5]:
BIG = 1e10
SMALL = 1e-12

In [6]:
def phi(t):
    """
    logistic function, returns 1 / (1 + exp(-t))
    """
    idx = t > 0
    out = np.empty(t.size, dtype=np.float)
    out[idx] = 1. / (1 + np.exp(-t[idx]))
    exp_t = np.exp(t[~idx])
    out[~idx] = exp_t / (1. + exp_t)
    return out

In [7]:
def log_logistic(t):
    """
    (minus) logistic loss function, returns log(1 / (1 + exp(-t)))
    """
    idx = t > 0
    out = np.zeros_like(t)
    out[idx] = np.log(1 + np.exp(-t[idx]))
    out[~idx] = (-t[~idx] + np.log(1 + np.exp(t[~idx])))
    return out

In [8]:
def ordinal_logistic_fit(X, y, alpha=0, l1_ratio=0, n_class=None, max_iter=10000,
                         verbose=False, solver='TNC', w0=None):
    """
    Ordinal logistic regression or proportional odds model.
    Uses scipy's optimize.fmin_slsqp solver.
    Parameters
    ----------
    X : {array, sparse matrix}, shape (n_samples, n_feaures)
        Input data
    y : array-like
        Target values
    max_iter : int
        Maximum number of iterations
    verbose: bool
        Print convergence information
    Returns
    -------
    w : array, shape (n_features,)
        coefficients of the linear model
    theta : array, shape (k,), where k is the different values of y
        vector of thresholds
    """

    X = np.asarray(X)
    y = np.asarray(y)
    w0 = None

    if not X.shape[0] == y.shape[0]:
        raise ValueError('Wrong shape for X and y')

    # .. order input ..
    idx = np.argsort(y)
    idx_inv = np.zeros_like(idx)
    idx_inv[idx] = np.arange(idx.size)
    X = X[idx]
    y = y[idx].astype(np.int)
    # make them continuous and start at zero
    unique_y = np.unique(y)
    for i, u in enumerate(unique_y):
        y[y == u] = i
    unique_y = np.unique(y)

    # .. utility arrays used in f_grad ..
    alpha = 0.
    k1 = np.sum(y == unique_y[0])
    E0 = (y[:, np.newaxis] == np.unique(y)).astype(np.int)
    E1 = np.roll(E0, -1, axis=-1)
    E1[:, -1] = 0.
    E0, E1 = map(sparse.csr_matrix, (E0.T, E1.T))

    def f_obj(x0, X, y):
        """
        Objective function
        """
        w, theta_0 = np.split(x0, [X.shape[1]])
        theta_1 = np.roll(theta_0, 1)
        t0 = theta_0[y]
        z = np.diff(theta_0)

        Xw = X.dot(w)
        a = t0 - Xw
        b = t0[k1:] - X[k1:].dot(w)
        c = (theta_1 - theta_0)[y][k1:]

        if np.any(c > 0):
            return BIG

        #loss = -(c[idx] + np.log(np.exp(-c[idx]) - 1)).sum()
        loss = -np.log(1 - np.exp(c)).sum()

        loss += b.sum() + log_logistic(b).sum() \
            + log_logistic(a).sum() \
            + .5 * alpha * w.dot(w) - np.log(z).sum()  # penalty
        if np.isnan(loss):
            pass
            #import ipdb; ipdb.set_trace()
        return loss

    def f_grad(x0, X, y):
        """
        Gradient of the objective function
        """
        w, theta_0 = np.split(x0, [X.shape[1]])
        theta_1 = np.roll(theta_0, 1)
        t0 = theta_0[y]
        t1 = theta_1[y]
        z = np.diff(theta_0)

        Xw = X.dot(w)
        a = t0 - Xw
        b = t0[k1:] - X[k1:].dot(w)
        c = (theta_1 - theta_0)[y][k1:]

        # gradient for w
        phi_a = phi(a)
        phi_b = phi(b)
        grad_w = -X[k1:].T.dot(phi_b) + X.T.dot(1 - phi_a) + alpha * w

        # gradient for theta
        idx = c > 0
        tmp = np.empty_like(c)
        tmp[idx] = 1. / (np.exp(-c[idx]) - 1)
        tmp[~idx] = np.exp(c[~idx]) / (1 - np.exp(c[~idx])) # should not need
        grad_theta = (E1 - E0)[:, k1:].dot(tmp) \
            + E0[:, k1:].dot(phi_b) - E0.dot(1 - phi_a)

        grad_theta[:-1] += 1. / np.diff(theta_0)
        grad_theta[1:] -= 1. / np.diff(theta_0)
        out = np.concatenate((grad_w, grad_theta))
        return out

    def f_hess(x0, s, X, y):
        x0 = np.asarray(x0)
        w, theta_0 = np.split(x0, [X.shape[1]])
        theta_1 = np.roll(theta_0, 1)
        t0 = theta_0[y]
        t1 = theta_1[y]
        z = np.diff(theta_0)

        Xw = X.dot(w)
        a = t0 - Xw
        b = t0[k1:] - X[k1:].dot(w)
        c = (theta_1 - theta_0)[y][k1:]

        D = np.diag(phi(a) * (1 - phi(a)))
        D_= np.diag(phi(b) * (1 - phi(b)))
        D1 = np.diag(np.exp(-c) / (np.exp(-c) - 1) ** 2)
        Ex = (E1 - E0)[:, k1:].toarray()
        Ex0 = E0.toarray()
        H_A = X[k1:].T.dot(D_).dot(X[k1:]) + X.T.dot(D).dot(X)
        H_C = - X[k1:].T.dot(D_).dot(E0[:, k1:].T.toarray()) \
            - X.T.dot(D).dot(E0.T.toarray())
        H_B = Ex.dot(D1).dot(Ex.T) + Ex0[:, k1:].dot(D_).dot(Ex0[:, k1:].T) \
            - Ex0.dot(D).dot(Ex0.T)

        p_w = H_A.shape[0]
        tmp0 = H_A.dot(s[:p_w]) + H_C.dot(s[p_w:])
        tmp1 = H_C.T.dot(s[:p_w]) + H_B.dot(s[p_w:])
        return np.concatenate((tmp0, tmp1))

        import ipdb; ipdb.set_trace()
        import pylab as pl
        pl.matshow(H_B)
        pl.colorbar()
        pl.title('True')
        import numdifftools as nd
        Hess = nd.Hessian(lambda x: f_obj(x, X, y))
        H = Hess(x0)
        pl.matshow(H[H_A.shape[0]:, H_A.shape[0]:])
        #pl.matshow()
        pl.title('estimated')
        pl.colorbar()
        pl.show()


    def grad_hess(x0, X, y):
        grad = f_grad(x0, X, y)
        hess = lambda x: f_hess(x0, x, X, y)
        return grad, hess

    x0 = np.random.randn(X.shape[1] + unique_y.size) / X.shape[1]
    if w0 is not None:
        x0[:X.shape[1]] = w0
    else:
        x0[:X.shape[1]] = 0.
    x0[X.shape[1]:] = np.sort(unique_y.size * np.random.rand(unique_y.size))

    #print('Check grad: %s' % optimize.check_grad(f_obj, f_grad, x0, X, y))
    #print(optimize.approx_fprime(x0, f_obj, 1e-6, X, y))
    #print(f_grad(x0, X, y))
    #print(optimize.approx_fprime(x0, f_obj, 1e-6, X, y) - f_grad(x0, X, y))
    #import ipdb; ipdb.set_trace()

    def callback(x0):
        x0 = np.asarray(x0)
        # print('Check grad: %s' % optimize.check_grad(f_obj, f_grad, x0, X, y))
        if verbose:
        # check that gradient is correctly computed
            print('OBJ: %s' % f_obj(x0, X, y))

    if solver == 'TRON':
        import pytron
        out = pytron.minimize(f_obj, grad_hess, x0, args=(X, y))
    else:
        options = {'maxiter' : max_iter, 'disp': 0, 'maxfun':10000}
        out = optimize.minimize(f_obj, x0, args=(X, y), method=solver,
            jac=f_grad, hessp=f_hess, options=options, callback=callback)

    if not out.success:
        warnings.warn(out.message)
    w, theta = np.split(out.x, [X.shape[1]])
    return w, theta

In [9]:
def ordinal_logistic_predict(w, theta, X):
    """
    Parameters
    ----------
    w : coefficients obtained by ordinal_logistic
    theta : thresholds
    """
    unique_theta = np.sort(np.unique(theta))
    out = X.dot(w)
    unique_theta[-1] = np.inf # p(y <= max_level) = 1
    tmp = out[:, None].repeat(unique_theta.size, axis=1)
    return np.argmax(tmp < unique_theta, axis=1)

----
----


In [10]:
#Function definitions

def onehot(df, target):
    #Returns an augmented dataframe that has appended one-hot columns and removed the original column\
    dumdum = pd.get_dummies(df[target], prefix=target, drop_first=True)
    df = pd.concat([df, dumdum], axis=1)
    df.drop(columns=target, inplace=True)
    return df

In [11]:
def autohot(df, target_list):
    #returns a dataframe that has one-hot encoded the target list and appended
    for feature in target_list:
        df = onehot(df, feature)
    return df

In [12]:
def drop_low_freqs(df, prefix_targets):
    #drops all columns from the target list whose sum <= 2
    target_list = []
    for prefix in prefix_targets:
        targets = [feature for feature in df.columns if prefix in feature]
        target_list += targets
    
    low_freqs = [col for col in target_list if sum(df[col]) <= 3]
    
    return df.drop(columns=low_freqs)

---
---

In [14]:
df = pd.read_csv("../model_ip.csv")

In [15]:
one_hots = ['year', 'a_country', 'b_country']
one_hots_min = ['year']


a_drops = ['dispnum', 'incidnum', 'y_duration', 'y_hiact', 'a_georegion', 'a_poliregion', 'b_georegion', 'b_poliregion', 'a_hiact']
a_drops_aggro = ['dispnum', 'incidnum', 'y_duration', 'y_hiact', 'a_country', 'b_country', 'a_georegion', 'a_poliregion', 'b_georegion', 'b_poliregion', 'a_hiact']

b_drops = ['dispnum', 'incidnum', 'y_duration', 'y_hiact', 'a_georegion', 'a_poliregion', 'b_georegion', 'b_poliregion', 'b_hiact']


a_target = 'a_hiact'
b_target = 'b_hiact'

---
---

In [16]:
a_prep = autohot(df, one_hots)
a_prep = drop_low_freqs(a_prep, one_hots)

a_pca = autohot(df, one_hots)
a_pca = drop_low_freqs(a_pca, one_hots)

a_prep_alt = autohot(df, one_hots_min)
a_prep_alt = drop_low_freqs(a_prep_alt, one_hots_min)

b_prep = autohot(df, one_hots)
b_prep = drop_low_freqs(b_prep, one_hots)

In [17]:
X_a = a_prep.drop(columns=a_drops)
y_a = a_prep[a_target]

X_pca = a_pca.drop(columns=a_drops)
y_pca = a_pca[a_target]

X_alt = a_prep_alt.drop(columns=a_drops_aggro)
y_alt = a_prep_alt[a_target]

X_b = b_prep.drop(columns=b_drops)
y_b = b_prep[b_target]

In [18]:
def get_ordinals(X, y):
    #automate train test split and return prediction MAE
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    
    w, theta = ordinal_logistic_fit(X_train, y_train)
    preds = ordinal_logistic_predict(w, theta, X_test)
    
    return y_test, preds


def make_whole(array):
    int_array = array
    for i in range(len(int_array)):
        int_array[i] = int(round(int_array[i]))
    return int_array

In [19]:
a_true, a_preds = get_ordinals(X_a, y_a)

/home/arcosion/anaconda3/lib/python3.7/site-packages/scipy/optimize/_minimize.py:529: RuntimeWarning: Method TNC does not use Hessian-vector product information (hessp).
  'information (hessp).' % method, RuntimeWarning)
/home/arcosion/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:190: UserWarning: Max. number of function evaluations reached
/home/arcosion/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  # This is added back by InteractiveShellApp.init_path()


In [20]:
b_true, b_preds = get_ordinals(X_b, y_b)

/home/arcosion/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:190: UserWarning: Max. number of function evaluations reached
/home/arcosion/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  # This is added back by InteractiveShellApp.init_path()


In [23]:
from sklearn.metrics import accuracy_score

a_preds = make_whole(a_preds)
b_preds = make_whole(b_preds)

print(f'MAE a_hiact:  {mean_absolute_error(y_true=a_true, y_pred=a_preds)}')
print(f'ACC a_hiact:  {round(accuracy_score(y_true=a_true, y_pred=a_preds), 3)*100}%')
print(f'MAE b_hiact:  {mean_absolute_error(y_true=b_true, y_pred=b_preds)}')
print(f'ACC b_hiact:  {round(accuracy_score(y_true=b_true, y_pred=b_preds), 3)*100}%')

MAE a_hiact:  2.418199419167473
ACC a_hiact:  28.000000000000004%
MAE b_hiact:  1.3010648596321395
ACC b_hiact:  63.9%


---
---
*Results were just okay... trying this in conjuction with proper PCA*

In [38]:
ss = StandardScaler()
pca = PCA()

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y_pca, random_state=42)

#standardize the data
X1_train = ss.fit_transform(X_train)
X1_test = ss.transform(X_test)

#apply PCA
Z_train = pca.fit_transform(X1_train)
Z_test = pca.transform(X1_test)

In [40]:
w, theta = ordinal_logistic_fit(Z_train, y_train)
preds = ordinal_logistic_predict(w, theta, Z_test)

mean_absolute_error(y_true=y_test, y_pred=preds)

/home/arcosion/anaconda3/lib/python3.7/site-packages/scipy/optimize/_minimize.py:529: RuntimeWarning: Method TNC does not use Hessian-vector product information (hessp).
  'information (hessp).' % method, RuntimeWarning)


2.42594385285576